In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import networkx as nx
import seaborn as sns
import pickle

In [ ]:
#임상데이터를 불러와서, 첫번째 열 제거하고, bcr_patient_barcode를 인덱스로 설정한다.

df_clinic = pd.read_csv('/home/lee/home/clinic_patient/BRCA_clinic_patient.csv')
df_clinic= df_clinic.iloc[:,1:]
df_clinic.set_index('bcr_patient_barcode', inplace=True)
df_clinic

In [ ]:
#분석하고자 하는 컬럼만 남긴다.
df_clinic = df_clinic[['icd_o_3_histology','icd_10','histological_type','lab_proc_her2_neu_immunohistochemistry_receptor_status',
'breast_carcinoma_progesterone_receptor_status','breast_carcinoma_estrogen_receptor_status']]


df_clinic

In [ ]:
#데이터프레임의 각 행을 입력받아 특정 값을 제외한 나머지 값을 dict형태로 변환한다.
# 함수는 row의 각 컬럼과 값을 순회하며 값이 NaN이 아니고 excluded_values 리스트에 없는 경우에만 col: val 쌍으로 포함한다.
def row_to_dict(row):
    #제외할 값 지정
    excluded_values = [
        '8523/3',
        '8524/3',
        '8010/3',
        'C50.9',
        'C50.919',
        'Other, specify',
        'Infiltrating Carcinoma NOS',
        'Equivocal',
        'Indeterminate'
        'Mixed Histology (please specify)'

          ]
    return {col: val for col, val in row.items() if pd.notna(val) and val not in excluded_values}

# 위 함수실행
row_sets = df_clinic.apply(row_to_dict, axis=1)

# 샘플 : {'컬럼이름': '값', '컬럼이름': '값', ... }
for index, value in row_sets.items():
    print(f'{index}: {value}')

In [ ]:

two_or_more_matches = [] #두개 이상 매치된 샘플 쌍 저장하는 리스트
no_matches = [] # 일치하는 항목이 두개 미만인 샘플 쌍 저장하는 리스트
matching_sametype_pairs = [] # 두 개 이상의 일치 항목을 가진 샘플 쌍을 별도로 저장함 (나중에 비교위해)

# 각샘플끼리 모두 비교
for (i1, row1), (i2, row2) in combinations(row_sets.items(), 2):
   
    common_items = set(row1.items()) & set(row2.items())
    
    
    num_matches = len(common_items)
    
    if num_matches >= 2:
        common_values = {k: v for k, v in common_items}
        two_or_more_matches.append((i1, i2, num_matches, common_values))
        matching_sametype_pairs.append((i1, i2))
    elif num_matches == 0:
        no_matches.append((i1, i2))

In [ ]:
print("2개이상 (같은 서브타입) :", len(two_or_more_matches))
print("2개미만 (다른 서브타입) :",len(no_matches))


In [ ]:
""""
 two_or_more_matches 리스트에 저장된 두 개 이상의 일치하는 항목이 있는 샘플 쌍을 출력
각 샘플 쌍의 인덱스, 일치하는 항목의 개수, 그리고 각 공통 항목의 열 이름과 값을 출력


print("Pairs with 2 or more matches:")
for pair in two_or_more_matches:
    print(f'{pair[0]} & {pair[1]}: {pair[2]} matches')
    for col, val in pair[3].items():
        print(f'  Column: {col}, Value: {val}')

"""

In [ ]:
# no_mathces리스트에 저장된 샘플 쌍을 출력
"""
print("\nPairs with no matches:")
for pair in no_matches:
    print(f'{pair[0]} & {pair[1]}')


"""

In [ ]:
import random

random.seed(42) 

#개수를 맞추기 위해 471863개에서 66696개만 랜덤으로 뽑았음
matching_sametype_pairs = random.sample(matching_sametype_pairs, min(66696, len(matching_sametype_pairs)))


print(len(matching_sametype_pairs))

In [ ]:
#그대로 사용
print(len(no_matches))

In [ ]:
# 피클에서 불러와서 네트워크 만들고 확인
with open('/home/lee/home/pizzz&pkl/use_pkl/pcc0.3_sil0.4_th0.1_pvalue0.05_BRCA.pkl', 'rb') as f:
    results = pickle.load(f)

print("Results loaded from clustering_results.pkl")

# 불러온 results를 사용하여 네트워크 구축 
G = nx.Graph()
for combo, result in results:
    if result is not None:
        for sample in result:
            # 샘플 이름을 12글자로 자름 (클리닉과 비교하기 위함)
            trimmed_samples = [s[:12] for s in sample[1]] 
            G.add_edge(combo[0], combo[1], sample=trimmed_samples)


# 네트워크 엣지 확인 (디버깅용)
print("Edges in the network:")
for u, v, data in G.edges(data=True):
    print(f'({u}, {v}): {data}')

In [ ]:
# 샘플별 유전자 집합 생성
sample_to_genes = {}
for u, v, data in G.edges(data=True):
    for sample in data['sample']:
        if sample not in sample_to_genes:
            sample_to_genes[sample] = set()
        sample_to_genes[sample].add(u)
        sample_to_genes[sample].add(v)


In [ ]:
# 자카드 계수 계산 함수
def jaccard_index(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# 자카드 계수 계산
sametype_jaccard = []
for pair in matching_sametype_pairs:
    try:
        set1 = sample_to_genes[pair[0]]
        set2 = sample_to_genes[pair[1]]
        jaccard = jaccard_index(set1, set2)
        sametype_jaccard.append(jaccard)
    except KeyError as e:#이거는 clinic에서는 있는 쌍인데, 우리가 구축한 네트워크엔 그 쌍이 없어서 출력되는 예외처리
        print(f"KeyError for pair {pair}: {e}") 

differenttype_jaccard = []
for pair in no_matches:
    try:
        set1 = sample_to_genes[pair[0]]
        set2 = sample_to_genes[pair[1]]
        jaccard = jaccard_index(set1, set2)
        differenttype_jaccard.append(jaccard)
    except KeyError as e:
        print(f"KeyError for pair {pair}: {e}")

In [ ]:
# 그래프 크기 설정
plt.figure(figsize=(14, 9))

# 서로 다른 서브타입 간 자카드 인덱스 히스토그램 추가
sns.histplot(sametype_jaccard, bins=20, kde=True, color='red', label='Between same Subtypes')
sns.histplot(differenttype_jaccard, bins=20, kde=True, color='blue', label='Between Different Subtypes')

# 0.5 축에 세로선 추가
plt.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Jaccard Index 0.5')

plt.xlim(0.1, 0.9)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.title('Distribution of Jaccard Index : BRCA')
plt.xlabel('Jaccard Index')
plt.ylabel('Count')
plt.legend(title='Subtype Jaccard Index', bbox_to_anchor=(1.05, 1), loc='upper left')

# 범례를 그래프 안으로 이동
plt.legend(title='Subtype Jaccard Index', loc='upper right', bbox_to_anchor=(0.95, 0.95), borderaxespad=0.)

plt.show()